In [ ]:
!pip install jsonlines

In [ ]:
import jsonlines
import json
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
import numpy as np
import re
import ast

def get_examples(file_path):
    with jsonlines.open(file_path) as reader:
        data = [line for line in reader]
    return data
def get_from_annotated_dataset(annotated_dataset,_id):
    for doc in annotated_dataset:
        if doc['doc_id'] == _id:
            return doc
annotated_dataset = get_examples('teste-progresso/annotations-medical_specialist-dpoc-bio-composed-multiple.jsonl')
df_data_info = pd.read_csv('test_data_info_no_short.csv')

#REMOVE ADDITIONAL BAD DATA
# df_data_info.drop(df_data_info[df_data_info['doc_id'] == '8396380d-e0b6-4b81-8fe9-0b99c611f9f3'].index, inplace=True)
# df_data_info.reset_index(drop=True,inplace=True)
def replace_substring(string, start, end, replacement):
    # Check if start and end are valid indices for the string
    if start < 0 or end > len(string) or start > end:
        return "Invalid start or end index"

    # Replace the substring from start to end with the replacement string
    new_string = string[:start] + replacement + string[end:]

    return new_string
def add_quotes(item):
    if (not (item.startswith('"') and item.endswith('"')) and not (item.startswith("'") and item.endswith("'"))):
        if (item.startswith('"') and not item.endswith('"')) or (item.startswith("'") and not item.endswith("'")):
            item = item.replace('"', '\\"').replace("'", "\\'")
    return item
def fix_quotes(text):
    # Regular expression to find lists and their items   
    pattern = r"""\[\s*((["'][^"']+["'])(?:\s+[^,]+)?)\s*,\s*(\[\s*["'][^"']+?["'](?:\s*,\s*["'][^"']+?["'])*\s*\])\s*\]"""
    annotation_list = {"annotations":[]}
    for match in re.finditer(pattern, text):        
        main_text = add_quotes(match.group(1))
        if main_text.startswith('"') and main_text.endswith('"') or (main_text.startswith("'") and main_text.endswith("'")):
            main_text = main_text[1:-1]
        current_list = [str(match.group(1)), ast.literal_eval(match.group(3))]
        current_list[0] = current_list[0][1:-1]
        annotation_list["annotations"].append(current_list)
    annotation_list = str(annotation_list)
    return annotation_list

def select_after_first_brace(string):
    pattern = r"(|[\'\"])(annotations)([\'\"]|)"
    
    matches = re.search(pattern, string)
    if matches != None:
        string = replace_substring(string, matches.span()[0], matches.span()[1], '"annotations"')
    string.replace("]\']",']]')
    string.replace("]\"]",']]')
    # print('STRING AFTER FIRST REGEX:', string)
    pattern = r'"annotations":\[.*\]\]'
    match = re.search(pattern, string)
    if match != None:
        # string = replace_substring(string, match.span()[0], match.span()[1], '"annotations":[')
        string = match.group(0)
        # print('STRING AFTER SECOND REGEX:', string)

    brace_index = string.find('"annotations"')
    string = '{'+string[brace_index:]
    # print('trying to select correct part of models output')
    # print(brace_index)
    # print(string)
    if brace_index == -1:
        return '{"annotations":[]}'
    else:
        if string[-3:] == "']]":
            string = string[:-3]+"']]]"
        if string.find("}") == -1:
            end_annotation_index = string.find(']]]')
            if string[-1:] != ']' and end_annotation_index != -1:                
                string = string[:end_annotation_index+3]+'}'
            elif string[-1:] != ']' and end_annotation_index == -1:
                end_annotation_index = string.find(']] ]')
                if end_annotation_index != -1:
                    string = string[:end_annotation_index+4]+'}'
                else:
                    string = string+']}'
            elif string[-7:].count(']') < 3:
                string = string+']}'
            else:
                string = string+'}'
        else:
            string = string[:string.find("}")+1]
        # print('results...')
        # print(string)


        string = fix_quotes(string)
        return string
# prediction_annotation = eval(model_response[0])
def prediction_to_labels(prediction_labels, data_info):
    if type(prediction_labels) == list:
        ze = prediction_labels[0]
        prediction_labels = ze
    prediction_labels = select_after_first_brace(prediction_labels)
    prediction_annotation = ast.literal_eval(prediction_labels)
    full_text = data_info['text']
    text_tokenized = data_info['labels']
    categorized_prediction = annotation_to_tokens(full_text, text_tokenized, prediction_annotation)
    labels = extract_labels_from_prediction(categorized_prediction)
    return labels
def truth_to_labels(data_info):
    labels = extract_labels_from_truth(data_info['labels'])
    return labels
def extract_labels_from_truth (data_info):
    text_tokenized = data_info
    categories = []
    for token in text_tokenized:
        if token[4] != None:
            categories.append(list(token[4].keys()))
        else:
            categories.append('0')
    return categories
def annotation_to_tokens (full_text, text_tokenized, prediction_annotation):
    clean_text_tokenized = [[token[0],token[1],token[2]] for token in text_tokenized]
    annotations = prediction_annotation['annotations']
    for annotation in annotations:
        # print('============= new annotation', annotation[0])
        start_pos = full_text.find(annotation[0])
        end_pos = len(annotation[0])+start_pos-1
        # print(f'end pos is {len(annotation)} + {start_pos} - 1 = {end_pos}')
        categorizing = False
        
        for token in clean_text_tokenized:
            # print(f'token pos {token[1]} annotation pos {start_pos} token {token[0]}')
            if token[1] == start_pos:
                # print('starting categorization...')
                # print(f'start pos {start_pos} end pos {end_pos} token {token[0]}')
                categorizing = True
            if categorizing:
                #adds category to token
                # print(f'adding category {annotation[1]} to token {token[0]}')
                token.append(annotation[1])
                if token[2] == end_pos:
                    # print(f'ending categorization at {token[0]}...')
                    categorizing = False
                    break
            
            # print(token)
    return clean_text_tokenized
def extract_labels_from_prediction (categorized_prediction):
    labels = []
    for token in categorized_prediction:
        if len(token) > 3:
            labels.append(token[3])
        else:
            labels.append('0')
    return labels

In [ ]:
# Convert labels of a annotation to binary vector
cats = {'pathophysiology':0,
        'etiology':1,
        'epidemiology':2,
        'history':3,
        'physical':4,
        'exams':5,
        'differential':6,
        'therapeutic':7}

def label2binary(labels):
    vet = [0] * 8
    # print(vet)
    for label in labels:
        if label in list(cats.keys()):
            vet[cats[label]] = 1
    return vet

# Convert all the text - divided in tokens - labels to binary vectors
def preprocess_classification(classi_data):
    # print('preprocess classification ======= ',classi_data)
    for index in range(len(classi_data)):
        classi_data[index] = label2binary(classi_data[index])
    return classi_data
# Convert all the text - divided in tokens - labels to boolean
def preprocess_annotation(ann_data):
    for index in range(len(ann_data)):
        ann_data[index] = 1 if sum(label2binary(ann_data[index])) >= 1 else 0
    return ann_data

def get_substrings_from_text(text):
    substring = text.split("\n")
    return substring

def find_complete_text_from_substring(substring, texts):
    complete_text = None
    count = 0
    for text in texts:
        if substring[0] in text['text']:
            count += 1
            complete_text = text
        elif substring[0][:100] in text['text']:
            count += 1
            complete_text = text
        elif substring[0].split('.')[0] in text['text']:
            count += 1
            complete_text = text
    if count == 1:
        return complete_text
    elif count > 1 and len(substring) > 1:
        substring.pop(0)
        find_complete_text_from_substring(substring, texts)
    else:
        print('not FOUND', substring[0][:60])
        return None

In [ ]:
rows_metrics_report = []

### Metrics for Experiment 1 - zero shot

In [ ]:
import json
import os
import ast


print('starting ',0,'shot ================================')
df_data_info = pd.read_csv('annotations_medical_specialist_pre_processed_no_short.csv')
row_metrics = [f'0_shot']
path = f'llama-outputs/full-dataset/no-short-data/temp-0.0/top-p-0.6/ideas-0-shot'
metrics_path = f'metrics-result/full-dataset/no-short-data/temp-0.0/top-p-0.6/ideas-0-shot'
file_names = [file for file in os.listdir(path) if file.endswith('.json')]
if not os.path.exists(metrics_path):
    os.makedirs(metrics_path)
    
exisiting_ids = []
df_data_info['truth'] = ''
df_data_info['truth_annotation'] = ''
df_data_info['prediction'] = ''
df_data_info['prediction_annotation'] = ''
for name in file_names:
    # print(name)
    llama_annotated = ''
    doc_id = name[-41:-5]
    if len(doc_id) == len('f98e69ee-fda6-4b1c-a8a9-c20b92630cb6'):
        truth_data = get_from_annotated_dataset(annotated_dataset, doc_id)
        with open(f'{path}/{name}', "r", encoding='utf8') as file:
            llama_annotated = json.load(file)
        pred_labels = prediction_to_labels(llama_annotated['response'],truth_data)
        truth_labels = truth_to_labels(truth_data)
        # print(doc_id)
        pred_classification_vector = preprocess_classification(pred_labels.copy())
        pred_general_annotation_vector = preprocess_annotation(pred_labels.copy())
        # print(pred_classification_vector)
        # print("==========")
        truth_classification_vector = preprocess_classification(truth_labels.copy())
        # print(truth_classification_vector)
        # print('######3')
        truth_general_annotation_vector = preprocess_annotation(truth_labels.copy())
        # print(truth_general_annotation_vector)
        

        df_data_info.loc[df_data_info['doc_id'] == doc_id, 'truth'] = str(truth_classification_vector)
        df_data_info.loc[df_data_info['doc_id'] == doc_id, 'truth_annotation'] = str(truth_general_annotation_vector)
        df_data_info.loc[df_data_info['doc_id'] == doc_id, 'prediction'] = str(pred_classification_vector)
        df_data_info.loc[df_data_info['doc_id'] == doc_id, 'prediction_annotation'] = str(pred_general_annotation_vector)
df_data_info['truth'] = list(df_data_info['truth'])
df_data_info['truth_annotation'] = df_data_info['truth_annotation']
df_data_info['prediction'] = df_data_info['prediction']
df_data_info['prediction_annotation'] = df_data_info['prediction_annotation']
# df_data_info['truth'] = df_data_info['truth'].str.strip('[]').str.split(',').map(np.array)
# df_data_info['truth_annotation'] = df_data_info['truth_annotation'].str.strip('[]').str.split(',').map(np.array)
# df_data_info['prediction'] = df_data_info['prediction'].str.strip('[]').str.split(',').map(np.array)
# df_data_info['prediction_annotation'] = df_data_info['prediction_annotation'].str.strip('[]').str.split(',').map(np.array)
df_data_info.to_csv(f'{metrics_path}/test_data_info_with_vectors.csv', index=False)

aux = []
aux2 = []
precision = []
recall = []
f1 = []

for index in tqdm(range(len(df_data_info))):
    truth = eval(df_data_info["truth"][index])
    prediction = eval(df_data_info["prediction"][index])
    truth_annotation = eval(df_data_info["truth_annotation"][index])
    prediction_annotation = eval(df_data_info["prediction_annotation"][index])

    value = jaccard_score((truth), (prediction), average="micro")
    aux.append([value,len(truth)])
    value2 = jaccard_score((truth_annotation), (prediction_annotation), average="micro")
    aux2.append([value2,len(truth)])
    
    # precision_now = precision_score(test_data3,test_data4,average='binary')

    # print(df_data_info["prediction_annotation"][index].split(','))
    precision_now = precision_score(truth_annotation, prediction_annotation,average='binary')
    precision.append([precision_now,len(truth)])

    recall_now = recall_score(truth_annotation, prediction_annotation,average='binary')
    recall.append([recall_now,len(truth)])

    f1_now = f1_score(truth_annotation, prediction_annotation,average='binary')
    f1.append([f1_now,len(truth)])

df_data_info["label_score"] = aux
df_data_info["annotation_score"] = aux2
df_data_info["precision"] = precision
df_data_info["recall"] = recall
df_data_info["f1"] = f1
df_data_info.to_csv(f'{metrics_path}/test_data_info_with_metrics.csv', index=False)

#Ponderada
score = 0
count = 0
for value in df_data_info["label_score"]:
    score += value[0] * value[1]
    count += value[1]

label_score_weight = score/count
row_metrics.append(label_score_weight)
# Mean

score = 0
count = 0
for value in df_data_info["label_score"]:
    score += value[0]
    count += 1

label_score_mean = score/count
row_metrics.append(label_score_mean)
# Ponderada

score = 0
count = 0
for value in df_data_info["annotation_score"]:
    score += value[0] * value[1]
    count += value[1]
    
annotation_score_weight = score/count
row_metrics.append(annotation_score_weight)
# Mean

score = 0
count = 0
for value in df_data_info["annotation_score"]:
    score += value[0]
    count += 1

annotation_score_mean = score/count
row_metrics.append(annotation_score_mean)
# Ponderada

score = 0
count = 0
for value in df_data_info["precision"]:
    score += value[0] * value[1]
    count += value[1]
    
precision_weight = score/count
row_metrics.append(precision_weight)
# Mean

score = 0
count = 0
for value in df_data_info["precision"]:
    score += value[0]
    count += 1

precision_mean = score/count
row_metrics.append(precision_mean)
# Ponderada

score = 0
count = 0
for value in df_data_info["recall"]:
    score += value[0] * value[1]
    count += value[1]
    
recall_weight = score/count
row_metrics.append(recall_weight)
# Mean

score = 0
count = 0
for value in df_data_info["recall"]:
    score += value[0]
    count += 1

recall_mean = score/count
row_metrics.append(recall_mean)
# Ponderada

score = 0
count = 0
for value in df_data_info["f1"]:
    score += value[0] * value[1]
    count += value[1]
    
f1_weight = score/count
row_metrics.append(f1_weight)
# Mean

score = 0
count = 0
for value in df_data_info["f1"]:
    score += value[0]
    count += 1

f1_mean = score/count
row_metrics.append(f1_mean)

weight_score = annotation_score_weight * label_score_weight
mean_score = annotation_score_mean * label_score_mean

row_metrics.append(weight_score)

row_metrics.append(mean_score)
rows_metrics_report.append(row_metrics)

### Metrics for Experiment 1 - Static Examples (1,2,3,4,10)

In [ ]:
import json
import os
import ast

for i_shot in [1,2,3,4,10]:
    print('starting ',i_shot,'shot ================================')
    df_data_info = pd.read_csv('annotations_medical_specialist_pre_processed_no_short.csv')
    row_metrics = [f'{i_shot}_shot_static']
    path = f'llama-outputs/full-dataset/no-short-data/temp-0.0/top-p-0.6/ideas-static-{i_shot}-shot'
    metrics_path = f'metrics-result/full-dataset/no-short-data/temp-0.0/top-p-0.6/ideas-static-{i_shot}-shot'
    file_names = [file for file in os.listdir(path) if file.endswith('.json')]
    if not os.path.exists(metrics_path):
        os.makedirs(metrics_path)
        
    exisiting_ids = []
    df_data_info['truth'] = ''
    df_data_info['truth_annotation'] = ''
    df_data_info['prediction'] = ''
    df_data_info['prediction_annotation'] = ''
    for name in file_names:
        # print(name)
        llama_annotated = ''
        doc_id = name[-41:-5]
        if len(doc_id) == len('f98e69ee-fda6-4b1c-a8a9-c20b92630cb6'):
            truth_data = get_from_annotated_dataset(annotated_dataset, doc_id)
            with open(f'{path}/{name}', "r", encoding='utf8') as file:
                llama_annotated = json.load(file)
            pred_labels = prediction_to_labels(llama_annotated['response'],truth_data)
            truth_labels = truth_to_labels(truth_data)
            # print(doc_id)
            pred_classification_vector = preprocess_classification(pred_labels.copy())
            pred_general_annotation_vector = preprocess_annotation(pred_labels.copy())
            # print(pred_classification_vector)
            # print("==========")
            truth_classification_vector = preprocess_classification(truth_labels.copy())
            # print(truth_classification_vector)
            # print('######3')
            truth_general_annotation_vector = preprocess_annotation(truth_labels.copy())
            # print(truth_general_annotation_vector)
            
    
            df_data_info.loc[df_data_info['doc_id'] == doc_id, 'truth'] = str(truth_classification_vector)
            df_data_info.loc[df_data_info['doc_id'] == doc_id, 'truth_annotation'] = str(truth_general_annotation_vector)
            df_data_info.loc[df_data_info['doc_id'] == doc_id, 'prediction'] = str(pred_classification_vector)
            df_data_info.loc[df_data_info['doc_id'] == doc_id, 'prediction_annotation'] = str(pred_general_annotation_vector)
    df_data_info['truth'] = list(df_data_info['truth'])
    df_data_info['truth_annotation'] = df_data_info['truth_annotation']
    df_data_info['prediction'] = df_data_info['prediction']
    df_data_info['prediction_annotation'] = df_data_info['prediction_annotation']
    # df_data_info['truth'] = df_data_info['truth'].str.strip('[]').str.split(',').map(np.array)
    # df_data_info['truth_annotation'] = df_data_info['truth_annotation'].str.strip('[]').str.split(',').map(np.array)
    # df_data_info['prediction'] = df_data_info['prediction'].str.strip('[]').str.split(',').map(np.array)
    # df_data_info['prediction_annotation'] = df_data_info['prediction_annotation'].str.strip('[]').str.split(',').map(np.array)
    df_data_info.to_csv(f'{metrics_path}/test_data_info_with_vectors.csv', index=False)
    
    aux = []
    aux2 = []
    precision = []
    recall = []
    f1 = []
    
    for index in tqdm(range(len(df_data_info))):
        truth = eval(df_data_info["truth"][index])
        prediction = eval(df_data_info["prediction"][index])
        truth_annotation = eval(df_data_info["truth_annotation"][index])
        prediction_annotation = eval(df_data_info["prediction_annotation"][index])
    
        value = jaccard_score((truth), (prediction), average="micro")
        aux.append([value,len(truth)])
        value2 = jaccard_score((truth_annotation), (prediction_annotation), average="micro")
        aux2.append([value2,len(truth)])
        
        # precision_now = precision_score(test_data3,test_data4,average='binary')
    
        # print(df_data_info["prediction_annotation"][index].split(','))
        precision_now = precision_score(truth_annotation, prediction_annotation,average='binary')
        precision.append([precision_now,len(truth)])
    
        recall_now = recall_score(truth_annotation, prediction_annotation,average='binary')
        recall.append([recall_now,len(truth)])
    
        f1_now = f1_score(truth_annotation, prediction_annotation,average='binary')
        f1.append([f1_now,len(truth)])
    
    df_data_info["label_score"] = aux
    df_data_info["annotation_score"] = aux2
    df_data_info["precision"] = precision
    df_data_info["recall"] = recall
    df_data_info["f1"] = f1
    df_data_info.to_csv(f'{metrics_path}/test_data_info_with_metrics.csv', index=False)

    #Ponderada
    score = 0
    count = 0
    for value in df_data_info["label_score"]:
        score += value[0] * value[1]
        count += value[1]
    
    label_score_weight = score/count
    row_metrics.append(label_score_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["label_score"]:
        score += value[0]
        count += 1
    
    label_score_mean = score/count
    row_metrics.append(label_score_mean)
    # Ponderada
    
    score = 0
    count = 0
    for value in df_data_info["annotation_score"]:
        score += value[0] * value[1]
        count += value[1]
        
    annotation_score_weight = score/count
    row_metrics.append(annotation_score_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["annotation_score"]:
        score += value[0]
        count += 1
    
    annotation_score_mean = score/count
    row_metrics.append(annotation_score_mean)
    # Ponderada
    
    score = 0
    count = 0
    for value in df_data_info["precision"]:
        score += value[0] * value[1]
        count += value[1]
        
    precision_weight = score/count
    row_metrics.append(precision_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["precision"]:
        score += value[0]
        count += 1
    
    precision_mean = score/count
    row_metrics.append(precision_mean)
    # Ponderada
    
    score = 0
    count = 0
    for value in df_data_info["recall"]:
        score += value[0] * value[1]
        count += value[1]
        
    recall_weight = score/count
    row_metrics.append(recall_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["recall"]:
        score += value[0]
        count += 1
    
    recall_mean = score/count
    row_metrics.append(recall_mean)
    # Ponderada
    
    score = 0
    count = 0
    for value in df_data_info["f1"]:
        score += value[0] * value[1]
        count += value[1]
        
    f1_weight = score/count
    row_metrics.append(f1_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["f1"]:
        score += value[0]
        count += 1
    
    f1_mean = score/count
    row_metrics.append(f1_mean)
    
    weight_score = annotation_score_weight * label_score_weight
    mean_score = annotation_score_mean * label_score_mean
    
    row_metrics.append(weight_score)
    
    row_metrics.append(mean_score)
    rows_metrics_report.append(row_metrics)

### Metrics for Experiment 2 - TF-IDF (1,2,3,4,10)

In [ ]:
import json
import os
import ast

for i_shot in [1,2,3,4,10]:
    print('starting ',i_shot,'shot ================================')
    df_data_info = pd.read_csv('annotations_medical_specialist_pre_processed_no_short.csv')
    row_metrics = [f'{i_shot}_shot_tf_id']

    path = f'llama-outputs/no-short-data/temp-0.0/top-p-0.6/ideas-tf-idf-{i_shot}-shot'
    metrics_path = f'metrics-result/no-short-data/temp-0.0/top-p-0.6/ideas-tf-idf-{i_shot}-shot'
    file_names = [file for file in os.listdir(path) if file.endswith('.json')]
    if not os.path.exists(metrics_path):
        os.makedirs(metrics_path)
        
    exisiting_ids = []
    df_data_info['truth'] = ''
    df_data_info['truth_annotation'] = ''
    df_data_info['prediction'] = ''
    df_data_info['prediction_annotation'] = ''
    for name in file_names:
        # print(name)
        llama_annotated = ''
        doc_id = name[-41:-5]
        if len(doc_id) == len('f98e69ee-fda6-4b1c-a8a9-c20b92630cb6'):
            truth_data = get_from_annotated_dataset(annotated_dataset, doc_id)
            with open(f'{path}/{name}', "r", encoding='utf8') as file:
                llama_annotated = json.load(file)
            pred_labels = prediction_to_labels(llama_annotated['response'],truth_data)
            truth_labels = truth_to_labels(truth_data)
            # print(doc_id)
            pred_classification_vector = preprocess_classification(pred_labels.copy())
            pred_general_annotation_vector = preprocess_annotation(pred_labels.copy())
            # print(pred_classification_vector)
            # print("==========")
            truth_classification_vector = preprocess_classification(truth_labels.copy())
            # print(truth_classification_vector)
            # print('######3')
            truth_general_annotation_vector = preprocess_annotation(truth_labels.copy())
            # print(truth_general_annotation_vector)
            
    
            df_data_info.loc[df_data_info['doc_id'] == doc_id, 'truth'] = str(truth_classification_vector)
            df_data_info.loc[df_data_info['doc_id'] == doc_id, 'truth_annotation'] = str(truth_general_annotation_vector)
            df_data_info.loc[df_data_info['doc_id'] == doc_id, 'prediction'] = str(pred_classification_vector)
            df_data_info.loc[df_data_info['doc_id'] == doc_id, 'prediction_annotation'] = str(pred_general_annotation_vector)
    df_data_info['truth'] = list(df_data_info['truth'])
    df_data_info['truth_annotation'] = df_data_info['truth_annotation']
    df_data_info['prediction'] = df_data_info['prediction']
    df_data_info['prediction_annotation'] = df_data_info['prediction_annotation']
    # df_data_info['truth'] = df_data_info['truth'].str.strip('[]').str.split(',').map(np.array)
    # df_data_info['truth_annotation'] = df_data_info['truth_annotation'].str.strip('[]').str.split(',').map(np.array)
    # df_data_info['prediction'] = df_data_info['prediction'].str.strip('[]').str.split(',').map(np.array)
    # df_data_info['prediction_annotation'] = df_data_info['prediction_annotation'].str.strip('[]').str.split(',').map(np.array)
    df_data_info.to_csv(f'{metrics_path}/test_data_info_with_vectors.csv', index=False)
    
    aux = []
    aux2 = []
    precision = []
    recall = []
    f1 = []
    
    for index in tqdm(range(len(df_data_info))):
        truth = eval(df_data_info["truth"][index])
        prediction = eval(df_data_info["prediction"][index])
        truth_annotation = eval(df_data_info["truth_annotation"][index])
        prediction_annotation = eval(df_data_info["prediction_annotation"][index])
    
        value = jaccard_score((truth), (prediction), average="micro")
        aux.append([value,len(truth)])
        value2 = jaccard_score((truth_annotation), (prediction_annotation), average="micro")
        aux2.append([value2,len(truth)])
        
        # precision_now = precision_score(test_data3,test_data4,average='binary')
    
        # print(df_data_info["prediction_annotation"][index].split(','))
        precision_now = precision_score(truth_annotation, prediction_annotation,average='binary')
        precision.append([precision_now,len(truth)])
    
        recall_now = recall_score(truth_annotation, prediction_annotation,average='binary')
        recall.append([recall_now,len(truth)])
    
        f1_now = f1_score(truth_annotation, prediction_annotation,average='binary')
        f1.append([f1_now,len(truth)])
    
    df_data_info["label_score"] = aux
    df_data_info["annotation_score"] = aux2
    df_data_info["precision"] = precision
    df_data_info["recall"] = recall
    df_data_info["f1"] = f1
    df_data_info.to_csv(f'{metrics_path}/test_data_info_with_metrics.csv', index=False)

    #Ponderada
    score = 0
    count = 0
    for value in df_data_info["label_score"]:
        score += value[0] * value[1]
        count += value[1]
    
    label_score_weight = score/count
    row_metrics.append(label_score_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["label_score"]:
        score += value[0]
        count += 1
    
    label_score_mean = score/count
    row_metrics.append(label_score_mean)
    # Ponderada
    
    score = 0
    count = 0
    for value in df_data_info["annotation_score"]:
        score += value[0] * value[1]
        count += value[1]
        
    annotation_score_weight = score/count
    row_metrics.append(annotation_score_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["annotation_score"]:
        score += value[0]
        count += 1
    
    annotation_score_mean = score/count
    row_metrics.append(annotation_score_mean)
    # Ponderada
    
    score = 0
    count = 0
    for value in df_data_info["precision"]:
        score += value[0] * value[1]
        count += value[1]
        
    precision_weight = score/count
    row_metrics.append(precision_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["precision"]:
        score += value[0]
        count += 1
    
    precision_mean = score/count
    row_metrics.append(precision_mean)
    # Ponderada
    
    score = 0
    count = 0
    for value in df_data_info["recall"]:
        score += value[0] * value[1]
        count += value[1]
        
    recall_weight = score/count
    row_metrics.append(recall_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["recall"]:
        score += value[0]
        count += 1
    
    recall_mean = score/count
    row_metrics.append(recall_mean)
    # Ponderada
    
    score = 0
    count = 0
    for value in df_data_info["f1"]:
        score += value[0] * value[1]
        count += value[1]
        
    f1_weight = score/count
    row_metrics.append(f1_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["f1"]:
        score += value[0]
        count += 1
    
    f1_mean = score/count
    row_metrics.append(f1_mean)
    
    weight_score = annotation_score_weight * label_score_weight
    mean_score = annotation_score_mean * label_score_mean
    
    row_metrics.append(weight_score)
    
    row_metrics.append(mean_score)
    rows_metrics_report.append(row_metrics)

### Metrics for Experiment 3 - TF-IDF Custom (similarity by text VS annotation) (1,2,3,4,10)

In [ ]:
import json
import os
import ast

for i_shot in [1,2,3,4,10]:
    print('starting ',i_shot,'shot ================================')
    df_data_info = pd.read_csv('annotations_medical_specialist_pre_processed_no_short.csv')
    row_metrics = [f'{i_shot}_shot_tf_id_custom']

    path = f'llama-outputs/no-short-data/temp-0.0/top-p-0.6/tf-idf-custom/ideas-tf-idf-{i_shot}-shot'
    metrics_path = f'metrics-result/no-short-data/temp-0.0/top-p-0.6/tf-idf-custom/ideas-tf-idf-{i_shot}-shot'
    file_names = [file for file in os.listdir(path) if file.endswith('.json')]
    if not os.path.exists(metrics_path):
        os.makedirs(metrics_path)
        
    exisiting_ids = []
    df_data_info['truth'] = ''
    df_data_info['truth_annotation'] = ''
    df_data_info['prediction'] = ''
    df_data_info['prediction_annotation'] = ''
    for name in file_names:
        # print(name)
        llama_annotated = ''
        doc_id = name[-41:-5]
        if len(doc_id) == len('f98e69ee-fda6-4b1c-a8a9-c20b92630cb6'):
            truth_data = get_from_annotated_dataset(annotated_dataset, doc_id)
            with open(f'{path}/{name}', "r", encoding='utf8') as file:
                llama_annotated = json.load(file)
            pred_labels = prediction_to_labels(llama_annotated['response'],truth_data)
            truth_labels = truth_to_labels(truth_data)
            # print(doc_id)
            pred_classification_vector = preprocess_classification(pred_labels.copy())
            pred_general_annotation_vector = preprocess_annotation(pred_labels.copy())
            # print(pred_classification_vector)
            # print("==========")
            truth_classification_vector = preprocess_classification(truth_labels.copy())
            # print(truth_classification_vector)
            # print('######3')
            truth_general_annotation_vector = preprocess_annotation(truth_labels.copy())
            # print(truth_general_annotation_vector)
            
    
            df_data_info.loc[df_data_info['doc_id'] == doc_id, 'truth'] = str(truth_classification_vector)
            df_data_info.loc[df_data_info['doc_id'] == doc_id, 'truth_annotation'] = str(truth_general_annotation_vector)
            df_data_info.loc[df_data_info['doc_id'] == doc_id, 'prediction'] = str(pred_classification_vector)
            df_data_info.loc[df_data_info['doc_id'] == doc_id, 'prediction_annotation'] = str(pred_general_annotation_vector)
    df_data_info['truth'] = list(df_data_info['truth'])
    df_data_info['truth_annotation'] = df_data_info['truth_annotation']
    df_data_info['prediction'] = df_data_info['prediction']
    df_data_info['prediction_annotation'] = df_data_info['prediction_annotation']
    # df_data_info['truth'] = df_data_info['truth'].str.strip('[]').str.split(',').map(np.array)
    # df_data_info['truth_annotation'] = df_data_info['truth_annotation'].str.strip('[]').str.split(',').map(np.array)
    # df_data_info['prediction'] = df_data_info['prediction'].str.strip('[]').str.split(',').map(np.array)
    # df_data_info['prediction_annotation'] = df_data_info['prediction_annotation'].str.strip('[]').str.split(',').map(np.array)
    df_data_info.to_csv(f'{metrics_path}/test_data_info_with_vectors.csv', index=False)
    
    aux = []
    aux2 = []
    precision = []
    recall = []
    f1 = []
    
    for index in tqdm(range(len(df_data_info))):
        truth = eval(df_data_info["truth"][index])
        prediction = eval(df_data_info["prediction"][index])
        truth_annotation = eval(df_data_info["truth_annotation"][index])
        prediction_annotation = eval(df_data_info["prediction_annotation"][index])
    
        value = jaccard_score((truth), (prediction), average="micro")
        aux.append([value,len(truth)])
        value2 = jaccard_score((truth_annotation), (prediction_annotation), average="micro")
        aux2.append([value2,len(truth)])
        
        # precision_now = precision_score(test_data3,test_data4,average='binary')
    
        # print(df_data_info["prediction_annotation"][index].split(','))
        precision_now = precision_score(truth_annotation, prediction_annotation,average='binary')
        precision.append([precision_now,len(truth)])
    
        recall_now = recall_score(truth_annotation, prediction_annotation,average='binary')
        recall.append([recall_now,len(truth)])
    
        f1_now = f1_score(truth_annotation, prediction_annotation,average='binary')
        f1.append([f1_now,len(truth)])
    
    df_data_info["label_score"] = aux
    df_data_info["annotation_score"] = aux2
    df_data_info["precision"] = precision
    df_data_info["recall"] = recall
    df_data_info["f1"] = f1
    df_data_info.to_csv(f'{metrics_path}/test_data_info_with_metrics.csv', index=False)

    #Ponderada
    score = 0
    count = 0
    for value in df_data_info["label_score"]:
        score += value[0] * value[1]
        count += value[1]
    
    label_score_weight = score/count
    row_metrics.append(label_score_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["label_score"]:
        score += value[0]
        count += 1
    
    label_score_mean = score/count
    row_metrics.append(label_score_mean)
    # Ponderada
    
    score = 0
    count = 0
    for value in df_data_info["annotation_score"]:
        score += value[0] * value[1]
        count += value[1]
        
    annotation_score_weight = score/count
    row_metrics.append(annotation_score_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["annotation_score"]:
        score += value[0]
        count += 1
    
    annotation_score_mean = score/count
    row_metrics.append(annotation_score_mean)
    # Ponderada
    
    score = 0
    count = 0
    for value in df_data_info["precision"]:
        score += value[0] * value[1]
        count += value[1]
        
    precision_weight = score/count
    row_metrics.append(precision_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["precision"]:
        score += value[0]
        count += 1
    
    precision_mean = score/count
    row_metrics.append(precision_mean)
    # Ponderada
    
    score = 0
    count = 0
    for value in df_data_info["recall"]:
        score += value[0] * value[1]
        count += value[1]
        
    recall_weight = score/count
    row_metrics.append(recall_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["recall"]:
        score += value[0]
        count += 1
    
    recall_mean = score/count
    row_metrics.append(recall_mean)
    # Ponderada
    
    score = 0
    count = 0
    for value in df_data_info["f1"]:
        score += value[0] * value[1]
        count += value[1]
        
    f1_weight = score/count
    row_metrics.append(f1_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["f1"]:
        score += value[0]
        count += 1
    
    f1_mean = score/count
    row_metrics.append(f1_mean)
    
    weight_score = annotation_score_weight * label_score_weight
    mean_score = annotation_score_mean * label_score_mean
    
    row_metrics.append(weight_score)
    
    row_metrics.append(mean_score)
    rows_metrics_report.append(row_metrics)

In [ ]:
import json
import os
import ast
row_metrics = ['one_shot_tf_idf_09_temp']

path = f'llama-outputs/no-short-data/temp-0.9/top-p-0.6/ideas-tf-idf-1-shot'
metrics_path = f'metrics-result/no-short-data/temp-0.9/top-p-0.6/ideas-tf-idf-1-shot'

if not os.path.exists(metrics_path):
    os.makedirs(metrics_path)

file_names = [file for file in os.listdir(path) if file.endswith('.json')]

exisiting_ids = []
df_data_info['truth'] = ''
df_data_info['truth_annotation'] = ''
df_data_info['prediction'] = ''
df_data_info['prediction_annotation'] = ''
for name in file_names:
    llama_annotated = ''
    doc_id = name[-41:-5]
    print(doc_id)
    if len(doc_id) == len('f98e69ee-fda6-4b1c-a8a9-c20b92630cb6'):
        truth_data = get_from_annotated_dataset(annotated_dataset, doc_id)
        with open(f'{path}/{name}', "r", encoding='utf8') as file:
            llama_annotated = json.load(file)
        pred_labels = prediction_to_labels(llama_annotated['response'],truth_data)
        truth_labels = truth_to_labels(truth_data)
        # print(doc_id)
        pred_classification_vector = preprocess_classification(pred_labels.copy())
        pred_general_annotation_vector = preprocess_annotation(pred_labels.copy())
        # print(pred_classification_vector)
        # print("==========")
        truth_classification_vector = preprocess_classification(truth_labels.copy())
        # print(truth_classification_vector)
        # print('######3')
        truth_general_annotation_vector = preprocess_annotation(truth_labels.copy())
        # print(truth_general_annotation_vector)

        df_data_info.loc[df_data_info['doc_id'] == doc_id, 'truth'] = str(truth_classification_vector)
        df_data_info.loc[df_data_info['doc_id'] == doc_id, 'truth_annotation'] = str(truth_general_annotation_vector)
        df_data_info.loc[df_data_info['doc_id'] == doc_id, 'prediction'] = str(pred_classification_vector)
        df_data_info.loc[df_data_info['doc_id'] == doc_id, 'prediction_annotation'] = str(pred_general_annotation_vector)
df_data_info['truth'] = list(df_data_info['truth'])
df_data_info['truth_annotation'] = df_data_info['truth_annotation']
df_data_info['prediction'] = df_data_info['prediction']
df_data_info['prediction_annotation'] = df_data_info['prediction_annotation']
# df_data_info['truth'] = df_data_info['truth'].str.strip('[]').str.split(',').map(np.array)
# df_data_info['truth_annotation'] = df_data_info['truth_annotation'].str.strip('[]').str.split(',').map(np.array)
# df_data_info['prediction'] = df_data_info['prediction'].str.strip('[]').str.split(',').map(np.array)
# df_data_info['prediction_annotation'] = df_data_info['prediction_annotation'].str.strip('[]').str.split(',').map(np.array)
df_data_info.to_csv(f'{metrics_path}/test_data_info_with_vectors.csv', index=False)


In [ ]:

aux = []
aux2 = []
precision = []
recall = []
f1 = []

for index in tqdm(range(len(df_data_info))):
    # if type(df_data_info["truth"][index]) != list:
    #     print(type(df_data_info["truth"][index]))
    #     print(type(eval(df_data_info["truth"][index])))
    #     print(eval(df_data_info["truth"][index]))
    #     df_data_info.loc[df_data_info["truth"][index], "truth"] = eval(df_data_info["truth"][index])
    # if type(df_data_info["prediction"][index]) != list:
    #     df_data_info.loc[df_data_info["prediction"][index], "prediction"] = eval(df_data_info["prediction"][index])
    # if type(df_data_info["truth_annotation"][index]) != list:
    #     df_data_info.loc[df_data_info["truth_annotation"][index], "truth_annotation"] = eval(df_data_info["truth_annotation"][index])
    # if type(df_data_info["prediction"][index]) != list:
    #     df_data_info.loc[df_data_info["prediction"][index], "prediction_annotation"] = eval(df_data_info["prediction_annotation"][index])
    
    truth = eval(df_data_info["truth"][index])
    prediction = eval(df_data_info["prediction"][index])
    truth_annotation = eval(df_data_info["truth_annotation"][index])
    prediction_annotation = eval(df_data_info["prediction_annotation"][index])

    value = jaccard_score((truth), (prediction), average="micro")
    aux.append([value,len(truth)])
    value2 = jaccard_score((truth_annotation), (prediction_annotation), average="micro")
    aux2.append([value2,len(truth)])
    
    # precision_now = precision_score(test_data3,test_data4,average='binary')

    # print(df_data_info["prediction_annotation"][index].split(','))
    precision_now = precision_score(truth_annotation, prediction_annotation,average='binary')
    precision.append([precision_now,len(truth)])

    recall_now = recall_score(truth_annotation, prediction_annotation,average='binary')
    recall.append([recall_now,len(truth)])

    f1_now = f1_score(truth_annotation, prediction_annotation,average='binary')
    f1.append([f1_now,len(truth)])

df_data_info["label_score"] = aux
df_data_info["annotation_score"] = aux2
df_data_info["precision"] = precision
df_data_info["recall"] = recall
df_data_info["f1"] = f1
df_data_info.head(5)

### Metrics for Experiment 5 - Random Examples (1,2,3,4,10)

In [ ]:
import json
import os
import ast

for i_shot in [1,2,3,4,10]:
    print('starting ',i_shot,'shot ================================')
    df_data_info = pd.read_csv('annotations_medical_specialist_pre_processed_no_short.csv')
    row_metrics = [f'{i_shot}_shot_random']
    path = f'llama-outputs/full-dataset/no-short-data/temp-0.0/top-p-0.6/random-shot-retrieval/ideas-random-{i_shot}-shot'
    metrics_path = f'metrics-result/full-dataset/no-short-data/temp-0.0/top-p-0.6/random-shot-retrieval/ideas-random-{i_shot}-shot'
    file_names = [file for file in os.listdir(path) if file.endswith('.json')]
    if not os.path.exists(metrics_path):
        os.makedirs(metrics_path)
        
    exisiting_ids = []
    df_data_info['truth'] = ''
    df_data_info['truth_annotation'] = ''
    df_data_info['prediction'] = ''
    df_data_info['prediction_annotation'] = ''
    for name in file_names:
        # print(name)
        llama_annotated = ''
        doc_id = name[-41:-5]
        if len(doc_id) == len('f98e69ee-fda6-4b1c-a8a9-c20b92630cb6'):
            truth_data = get_from_annotated_dataset(annotated_dataset, doc_id)
            with open(f'{path}/{name}', "r", encoding='utf8') as file:
                llama_annotated = json.load(file)
            pred_labels = prediction_to_labels(llama_annotated['response'],truth_data)
            truth_labels = truth_to_labels(truth_data)
            # print(doc_id)
            pred_classification_vector = preprocess_classification(pred_labels.copy())
            pred_general_annotation_vector = preprocess_annotation(pred_labels.copy())
            # print(pred_classification_vector)
            # print("==========")
            truth_classification_vector = preprocess_classification(truth_labels.copy())
            # print(truth_classification_vector)
            # print('######3')
            truth_general_annotation_vector = preprocess_annotation(truth_labels.copy())
            # print(truth_general_annotation_vector)
            
    
            df_data_info.loc[df_data_info['doc_id'] == doc_id, 'truth'] = str(truth_classification_vector)
            df_data_info.loc[df_data_info['doc_id'] == doc_id, 'truth_annotation'] = str(truth_general_annotation_vector)
            df_data_info.loc[df_data_info['doc_id'] == doc_id, 'prediction'] = str(pred_classification_vector)
            df_data_info.loc[df_data_info['doc_id'] == doc_id, 'prediction_annotation'] = str(pred_general_annotation_vector)
    df_data_info['truth'] = list(df_data_info['truth'])
    df_data_info['truth_annotation'] = df_data_info['truth_annotation']
    df_data_info['prediction'] = df_data_info['prediction']
    df_data_info['prediction_annotation'] = df_data_info['prediction_annotation']
    # df_data_info['truth'] = df_data_info['truth'].str.strip('[]').str.split(',').map(np.array)
    # df_data_info['truth_annotation'] = df_data_info['truth_annotation'].str.strip('[]').str.split(',').map(np.array)
    # df_data_info['prediction'] = df_data_info['prediction'].str.strip('[]').str.split(',').map(np.array)
    # df_data_info['prediction_annotation'] = df_data_info['prediction_annotation'].str.strip('[]').str.split(',').map(np.array)
    df_data_info.to_csv(f'{metrics_path}/test_data_info_with_vectors.csv', index=False)
    
    aux = []
    aux2 = []
    precision = []
    recall = []
    f1 = []
    
    for index in tqdm(range(len(df_data_info))):
        truth = eval(df_data_info["truth"][index])
        prediction = eval(df_data_info["prediction"][index])
        truth_annotation = eval(df_data_info["truth_annotation"][index])
        prediction_annotation = eval(df_data_info["prediction_annotation"][index])
    
        value = jaccard_score((truth), (prediction), average="micro")
        aux.append([value,len(truth)])
        value2 = jaccard_score((truth_annotation), (prediction_annotation), average="micro")
        aux2.append([value2,len(truth)])
        
        # precision_now = precision_score(test_data3,test_data4,average='binary')
    
        # print(df_data_info["prediction_annotation"][index].split(','))
        precision_now = precision_score(truth_annotation, prediction_annotation,average='binary')
        precision.append([precision_now,len(truth)])
    
        recall_now = recall_score(truth_annotation, prediction_annotation,average='binary')
        recall.append([recall_now,len(truth)])
    
        f1_now = f1_score(truth_annotation, prediction_annotation,average='binary')
        f1.append([f1_now,len(truth)])
    
    df_data_info["label_score"] = aux
    df_data_info["annotation_score"] = aux2
    df_data_info["precision"] = precision
    df_data_info["recall"] = recall
    df_data_info["f1"] = f1
    df_data_info.to_csv(f'{metrics_path}/test_data_info_with_metrics.csv', index=False)

    #Ponderada
    score = 0
    count = 0
    for value in df_data_info["label_score"]:
        score += value[0] * value[1]
        count += value[1]
    
    label_score_weight = score/count
    row_metrics.append(label_score_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["label_score"]:
        score += value[0]
        count += 1
    
    label_score_mean = score/count
    row_metrics.append(label_score_mean)
    # Ponderada
    
    score = 0
    count = 0
    for value in df_data_info["annotation_score"]:
        score += value[0] * value[1]
        count += value[1]
        
    annotation_score_weight = score/count
    row_metrics.append(annotation_score_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["annotation_score"]:
        score += value[0]
        count += 1
    
    annotation_score_mean = score/count
    row_metrics.append(annotation_score_mean)
    # Ponderada
    
    score = 0
    count = 0
    for value in df_data_info["precision"]:
        score += value[0] * value[1]
        count += value[1]
        
    precision_weight = score/count
    row_metrics.append(precision_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["precision"]:
        score += value[0]
        count += 1
    
    precision_mean = score/count
    row_metrics.append(precision_mean)
    # Ponderada
    
    score = 0
    count = 0
    for value in df_data_info["recall"]:
        score += value[0] * value[1]
        count += value[1]
        
    recall_weight = score/count
    row_metrics.append(recall_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["recall"]:
        score += value[0]
        count += 1
    
    recall_mean = score/count
    row_metrics.append(recall_mean)
    # Ponderada
    
    score = 0
    count = 0
    for value in df_data_info["f1"]:
        score += value[0] * value[1]
        count += value[1]
        
    f1_weight = score/count
    row_metrics.append(f1_weight)
    # Mean
    
    score = 0
    count = 0
    for value in df_data_info["f1"]:
        score += value[0]
        count += 1
    
    f1_mean = score/count
    row_metrics.append(f1_mean)
    
    weight_score = annotation_score_weight * label_score_weight
    mean_score = annotation_score_mean * label_score_mean
    
    row_metrics.append(weight_score)
    
    row_metrics.append(mean_score)
    rows_metrics_report.append(row_metrics)

## Save and compress metrics

In [ ]:
rows_metrics_report

In [ ]:

 
#Create a DataFrame object
df_metrics = pd.DataFrame(rows_metrics_report,
                  columns = ['approach','class_weight' , 'class_mean', 'ann_weight' , 'ann_mean','precision_weight','precision_mean','recall_weight','recall_mean','f1_weight','f1_mean','global_weight','global_mean'])

In [ ]:
df_metrics.to_csv('df_metrics_no_short_random.csv',index=False)

In [ ]:
import tarfile
import os

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))


In [ ]:
make_tarfile('metrics_result.tar.gz','metrics-result')


In [ ]:
make_tarfile('llama-outputs.tar.gz','llama-outputs')

In [ ]:
make_tarfile('data_for_metrics.tar.gz','data_for_metrics')

In [ ]:
make_tarfile('metrics-token.tar.gz','metrics-token')

In [ ]:
make_tarfile('teste-progresso.tar.gz','teste-progresso')

In [ ]:
make_tarfile('llama-outputs_10_shot_custom_temp_0.tar.gz','llama-outputs/full-dataset/no-short-data/temp-0.0/top-p-0.6/tf-idf-custom/ideas-tf-idf-10-shot')

In [ ]:
make_tarfile('llama-outputs_10_shot_custom_temp_0.9.tar.gz','llama-outputs/full-dataset/no-short-data/temp-0.9/top-p-0.6/tf-idf-custom/ideas-tf-idf-10-shot')

In [ ]:
make_tarfile('llama-outputs_10_shot_custom_temp_0.0.tar.gz','llama-outputs/full-dataset/no-short-data/temp-0.0/top-p-0.6/ideas-tf-idf-10-shot')

In [ ]:
make_tarfile('llama-outputs_0_shot_with_detail_no_explanation.tar.gz','llama-outputs/full-dataset/overall-organization/temp-0.0/top-p-0.6/with_detail_no_explanation')

In [ ]:
make_tarfile('llama-outputs_0_shot_with_detail_with_explanation.tar.gz','llama-outputs/full-dataset/overall-organization/temp-0.0/top-p-0.6/with_detail_with_explanation')

In [ ]:
make_tarfile('llama-outputs-augmented.tar.gz','llama-outputs-augmented')

In [ ]:
make_tarfile('metrics-result-augmented.tar.gz','metrics-result-augmented')